In [ ]:
import warnings, os
import datetime as dt
import itertools as it
from numpy import nan, inf
import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

# Load

In [ ]:
Raw = pd.read_csv('Storage/Metatrader_WINN.csv', parse_dates=['datetime'])
Raw.head()

In [ ]:
def QUERY_SELECT(Raw, SYMBOLS, TFRAMES):
    return Raw[Raw['symbol'].isin(SYMBOLS) & Raw['tf'].isin(TFRAMES)] .reset_index(drop=1)

Query = QUERY_SELECT(Raw, SYMBOLS=['WIN$N'], TFRAMES=['H1'])
Raw.head()

# Data

In [ ]:
def FORMAT_SOURCE(Query):
    Src = pd.DataFrame()
    Src['A']        = Query['a']
    Src['Z']        = Query['z']
    Src['Symbol']   = Query['symbol']
    Src['TF']       = Query['tf']
    
    Src['Datetime'] = Query['datetime']
    Src['Date']     = Query['datetime'].dt.date
    Src['Time']     = Query['datetime'].dt.time
    
    Src['Ticks']    = Query['tick_volume']
    Src['Volume']   = Query['real_volume']
    Src['Price']    = Query['close']
    
    Src['Open']     = Query['open']
    Src['High']     = Query['high']
    Src['Low']      = Query['low']
    Src['Close']    = Query['close']

    Src['Change']       = Src['Close'] - Src['Open']
    Src['Hilo']         = Src['High']  - Src['Low']

    Src['Sign']         = Src['Change'].apply(lambda x: -1 if (x <  0) else  +1)
    Src['Sign Bull']    = Src['Change'].apply(lambda x: +1 if (x >= 0) else nan)
    Src['Sign Bear']    = Src['Change'].apply(lambda x: -1 if (x <  0) else nan)
    return Src 

Src = FORMAT_SOURCE(Query)
Src.head()

In [ ]:
def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc['Chg Abs'] = Calc['Change'].abs()
    Calc['Chg Pos'] = Calc['Chg Abs'] * Calc['Sign Bull']
    Calc['Chg Neg'] = Calc['Chg Abs'] * Calc['Sign Bear']

    Calc['HL Pos']  = Calc['Hilo'] * Calc['Sign Bull']
    Calc['HL Neg']  = Calc['Hilo'] * Calc['Sign Bear']
    return Calc

Calc = CALCULATIONS(Src)
Calc.head()

# Stats

In [14]:
Calc.groupby('Time')[['Change']].describe().round(0).astype(int)

Change                                    
          count mean  std   min  25% 50%  75%   max
Time                                               
09:00:00   1240   -7  479 -2355 -290  10  280  2045
10:00:00   1240   12  599 -2340 -355 -10  380  4280
11:00:00   1240  -31  495 -2230 -345 -18  270  2625
12:00:00   1240   12  409 -1885 -216  18  250  1675
13:00:00   1245    1  309 -1370 -170   0  185  1045
14:00:00   1245    0  355 -2290 -185   5  195  3985
15:00:00   1245   -8  363 -2875 -185  -5  190  1850
16:00:00   1245    5  373 -1935 -165  10  180  3710
17:00:00   1245  -13  300 -1690 -165  -5  145  1280
18:00:00    682   13  220 -1705 -110  15  130   755

In [15]:
Calc.groupby('Time')[['Chg Abs']].describe().round(0).astype(int)

Chg Abs                                   
           count mean  std min  25%  50%  75%   max
Time                                               
09:00:00    1240  364  312   0  135  290  515  2355
10:00:00    1240  462  381   0  180  375  656  4280
11:00:00    1240  380  319   0  149  300  521  2625
12:00:00    1240  307  270   0  115  235  415  1885
13:00:00    1245  231  205   0   75  180  320  1370
14:00:00    1245  248  254   0   75  190  330  3985
15:00:00    1245  253  260   0   75  185  335  2875
16:00:00    1245  251  276   0   80  170  320  3710
17:00:00    1245  214  211   0   75  150  285  1690
18:00:00     682  158  153   0   55  125  214  1705

In [16]:
Calc.groupby('Time')[['Chg Pos']].describe().round(0).astype(int)

Chg Pos                                   
           count mean  std min  25%  50%  75%   max
Time                                               
09:00:00     636  348  301   0  115  270  498  2045
10:00:00     615  478  402   0  198  380  670  4280
11:00:00     605  357  311   0  140  275  480  2625
12:00:00     638  310  270   0  120  240  419  1675
13:00:00     625  232  195   0   80  185  330  1045
14:00:00     637  242  269   0   70  185  320  3985
15:00:00     617  247  239   0   70  190  340  1850
16:00:00     650  245  295   0   80  165  300  3710
17:00:00     620  202  194   0   74  145  275  1280
18:00:00     368  158  139   0   55  120  225   755

In [17]:
Calc.groupby('Time')[['Chg Neg']].describe().round(0).astype(int)

Chg Neg                                   
           count mean  std   min  25%  50%  75% max
Time                                               
09:00:00     604 -381  323 -2355 -530 -300 -150  -5
10:00:00     625 -447  359 -2340 -630 -355 -170  -5
11:00:00     635 -401  326 -2230 -545 -335 -160  -5
12:00:00     602 -304  271 -1885 -415 -225 -115  -5
13:00:00     620 -231  215 -1370 -315 -172  -75  -5
14:00:00     608 -253  237 -2290 -340 -190  -80  -5
15:00:00     628 -259  279 -2875 -320 -180  -80  -5
16:00:00     595 -257  253 -1935 -355 -175  -80  -5
17:00:00     625 -226  226 -1690 -285 -165  -75  -5
18:00:00     314 -157  168 -1705 -205 -125  -55  -5

In [18]:
Calc.groupby('Time')[['Hilo']].describe().round(0).astype(int)

Hilo                                     
         count mean  std  min  25%  50%   75%   max
Time                                               
09:00:00  1240  745  332  215  505  670   905  2965
10:00:00  1240  912  370  260  659  840  1080  4780
11:00:00  1240  771  333  255  540  715   925  3655
12:00:00  1240  618  295  180  410  550   756  2325
13:00:00  1245  505  239  125  340  445   610  2235
14:00:00  1245  526  314  140  345  455   620  6160
15:00:00  1245  537  319  120  330  460   640  3380
16:00:00  1245  514  349  105  315  430   610  5225
17:00:00  1245  431  267   90  265  370   490  2420
18:00:00   682  293  182   75  185  250   345  2290

In [19]:
Calc.groupby('Time')[['HL Pos']].describe().round(0).astype(int)

HL Pos                                     
          count mean  std  min  25%  50%   75%   max
Time                                                
09:00:00    636  736  317  215  500  670   895  2355
10:00:00    615  925  385  285  682  855  1080  4780
11:00:00    605  749  329  255  520  700   895  3655
12:00:00    638  617  293  180  410  545   754  2295
13:00:00    625  501  229  125  345  460   600  2210
14:00:00    637  515  347  150  335  445   595  6160
15:00:00    617  531  318  120  330  465   625  2850
16:00:00    650  513  393  135  315  420   585  5225
17:00:00    620  411  224  110  265  358   480  1625
18:00:00    368  293  158   75  180  250   350  1020

In [20]:
Calc.groupby('Time')[['HL Neg']].describe().round(0).astype(int)

HL Neg                                     
          count mean  std   min   25%  50%  75%  max
Time                                                
09:00:00    604 -754  347 -2965  -911 -670 -510 -275
10:00:00    625 -898  354 -2645 -1075 -820 -640 -260
11:00:00    635 -792  336 -2875  -942 -730 -565 -260
12:00:00    602 -620  298 -2325  -764 -555 -406 -185
13:00:00    620 -508  249 -2235  -615 -440 -340 -155
14:00:00    608 -538  274 -2615  -640 -470 -355 -140
15:00:00    628 -543  320 -3380  -650 -450 -334 -140
16:00:00    595 -516  294 -2420  -625 -440 -318 -105
17:00:00    625 -451  303 -2420  -510 -370 -270  -90
18:00:00    314 -293  206 -2290  -340 -245 -186  -95

# Snippets

In [21]:
def FORMAT_SOURCE(Query):
    Src = pd.DataFrame()
    Src['Time']      = Query['datetime'].dt.time
    Src['Change']    = Src['Close'] - Src['Open']
    Src['Hilo']      = Src['High']  - Src['Low']
    Src['Sign']      = Src['Change'].apply(lambda x: -1 if (x <  0) else  +1)
    Src['Sign Bull'] = Src['Change'].apply(lambda x: +1 if (x >= 0) else nan)
    Src['Sign Bear'] = Src['Change'].apply(lambda x: -1 if (x <  0) else nan)
    return Src


def CALCULATIONS(Src):
    Calc = pd.DataFrame(Src)
    Calc['Chg Abs'] = Calc['Change'].abs()
    Calc['Chg Pos'] = Calc['Chg Abs'] * Calc['Sign Bull']
    Calc['Chg Neg'] = Calc['Chg Abs'] * Calc['Sign Bear']

    Calc['HL Pos']  = Calc['Hilo'] * Calc['Sign Bull']
    Calc['HL Neg']  = Calc['Hilo'] * Calc['Sign Bear']
    return Calc


Calc.groupby('Time')[['Hilo']].describe().round(0).astype(int)
Calc.groupby('Time')[['HL Pos']].describe().round(0).astype(int)
Calc.groupby('Time')[['HL Neg']].describe().round(0).astype(int)

Calc.groupby('Time')[['Change']].describe().round(0).astype(int)
Calc.groupby('Time')[['Chg Abs']].describe().round(0).astype(int)
Calc.groupby('Time')[['Chg Pos']].describe().round(0).astype(int)
Calc.groupby('Time')[['Chg Neg']].describe().round(0).astype(int)

Chg Neg                                   
           count mean  std   min  25%  50%  75% max
Time                                               
09:00:00     604 -381  323 -2355 -530 -300 -150  -5
10:00:00     625 -447  359 -2340 -630 -355 -170  -5
11:00:00     635 -401  326 -2230 -545 -335 -160  -5
12:00:00     602 -304  271 -1885 -415 -225 -115  -5
13:00:00     620 -231  215 -1370 -315 -172  -75  -5
14:00:00     608 -253  237 -2290 -340 -190  -80  -5
15:00:00     628 -259  279 -2875 -320 -180  -80  -5
16:00:00     595 -257  253 -1935 -355 -175  -80  -5
17:00:00     625 -226  226 -1690 -285 -165  -75  -5
18:00:00     314 -157  168 -1705 -205 -125  -55  -5